In [3]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection string
engine = create_engine('postgresql://uvg_user:uvg_password@db:5432/health_data')

# Simple query to test the foundation layer
try:
    df = pd.read_sql("SELECT 1 as connection_status", engine)
    print("Connection Successful! Your Biomedical Data Stack is ready.")
    print(df)
except Exception as e:
    print(f"Connection Failed: {e}")

Connection Successful! Your Biomedical Data Stack is ready.
   connection_status
0                  1


In [ ]:
query = """
-- escribe tu SQL aquí
"""
df = pd.read_sql(query, engine)
df

In [4]:
pd.read_sql("""
SELECT
  p.subject_id,
  p.external_id,
  p.full_name,
  a.hadm_id,
  a.admission_type,
  a.admittime,
  a.dischtime,
  a.hospital_expire_flag
FROM patients p
JOIN admissions a ON a.subject_id = p.subject_id
ORDER BY a.admittime;
""", engine)

,subject_id,external_id,full_name,hadm_id,admission_type,admittime,dischtime,hospital_expire_flag
0,1,MRN-0001,Ana L??pez,1,Emergency,2101-01-10 08:00:00,2101-01-15 14:00:00,False
1,6,MRN-0006,Luc??a Herrera,7,Emergency,2101-02-05 09:30:00,2101-02-08 10:00:00,False
2,2,MRN-0002,Carlos P??rez,3,Emergency,2101-03-20 22:00:00,2101-03-28 10:00:00,True
3,7,MRN-0007,Miguel Castillo,9,Emergency,2101-05-14 07:00:00,2101-05-20 15:00:00,False
4,3,MRN-0003,Mar??a G??mez,4,Urgent,2101-07-11 13:00:00,2101-07-14 11:00:00,False
5,8,MRN-0008,Sof??a Morales,10,Emergency,2101-08-21 20:00:00,2101-08-24 09:00:00,False
6,4,MRN-0004,Jos?? Mart??nez,5,Emergency,2101-09-02 06:00:00,2101-09-10 15:00:00,False
7,6,MRN-0006,Luc??a Herrera,8,Urgent,2101-11-01 16:00:00,2101-11-04 12:00:00,False
8,5,MRN-0005,Alex Rivera,6,Emergency,2101-12-18 19:00:00,2101-12-22 08:00:00,False
9,1,MRN-0001,Ana L??pez,2,Elective,2102-06-01 10:00:00,2102-06-05 09:00:00,False


In [5]:
pd.read_sql("""
SELECT
  p.external_id,
  p.full_name,
  COUNT(a.hadm_id) AS n_admissions
FROM patients p
JOIN admissions a
  ON p.subject_id = a.subject_id
GROUP BY
  p.subject_id,
  p.external_id,
  p.full_name
HAVING COUNT(a.hadm_id) > 1
ORDER BY n_admissions DESC;
""", engine)

,external_id,full_name,n_admissions
0,MRN-0006,Luc??a Herrera,2
1,MRN-0001,Ana L??pez,2


In [7]:
pd.read_sql("""
SELECT
  a.hadm_id,
  p.external_id,
  p.full_name,
  EXTRACT(EPOCH FROM (a.dischtime - a.admittime)) / 86400.0
    AS length_of_stay_days
FROM patients p
JOIN admissions a
  ON p.subject_id = a.subject_id
ORDER BY length_of_stay_days DESC;
""", engine)

,hadm_id,external_id,full_name,length_of_stay_days
0,5,MRN-0004,Jos?? Mart??nez,8.375000
1,3,MRN-0002,Carlos P??rez,7.500000
2,9,MRN-0007,Miguel Castillo,6.333333
3,1,MRN-0001,Ana L??pez,5.250000
4,2,MRN-0001,Ana L??pez,3.958333
5,6,MRN-0005,Alex Rivera,3.541667
6,7,MRN-0006,Luc??a Herrera,3.020833
7,4,MRN-0003,Mar??a G??mez,2.916667
8,8,MRN-0006,Luc??a Herrera,2.833333
9,10,MRN-0008,Sof??a Morales,2.541667


In [8]:
pd.read_sql("""
SELECT
  a.hadm_id,
  p.external_id,
  p.full_name,
  MAX(le.value_num) AS max_creatinine
FROM labevents le
JOIN d_labitems d
  ON le.labitem_id = d.labitem_id
JOIN admissions a
  ON le.hadm_id = a.hadm_id
JOIN patients p
  ON a.subject_id = p.subject_id
WHERE d.label = 'Creatinine'
GROUP BY
  a.hadm_id,
  p.external_id,
  p.full_name
ORDER BY max_creatinine DESC;
""", engine)

,hadm_id,external_id,full_name,max_creatinine
0,3,MRN-0002,Carlos P??rez,3.1
1,9,MRN-0007,Miguel Castillo,1.8
2,1,MRN-0001,Ana L??pez,1.6
3,8,MRN-0006,Luc??a Herrera,1.4
4,7,MRN-0006,Luc??a Herrera,0.9


In [9]:
pd.read_sql( """
SELECT 
    a.hadm_id, 
    p.external_id, 
    p.full_name,
    MAX(CASE WHEN dl.label = 'White Blood Cells' THEN le.value_num END) AS max_wbc,
    MAX(CASE WHEN dl.label = 'Lactate' THEN le.value_num END) AS max_lactate
FROM patients p
JOIN admissions a ON p.subject_id = a.subject_id
JOIN labevents le ON a.hadm_id = le.hadm_id
JOIN d_labitems dl ON le.labitem_id = dl.labitem_id
WHERE dl.label IN ('White Blood Cells', 'Lactate')
GROUP BY a.hadm_id, p.external_id, p.full_name
HAVING 
    MAX(CASE WHEN dl.label = 'White Blood Cells' THEN le.value_num END) > 12 
    OR 
    MAX(CASE WHEN dl.label = 'Lactate' THEN le.value_num END) > 2
ORDER BY max_lactate DESC;
""", engine)

,hadm_id,external_id,full_name,max_wbc,max_lactate
0,6,MRN-0005,Alex Rivera,12.5,NaN
1,10,MRN-0008,Sof??a Morales,13.0,NaN
2,3,MRN-0002,Carlos P??rez,18.4,3.8
3,5,MRN-0004,Jos?? Mart??nez,14.2,2.2


## Respuestas a las preguntas del Laboratorio
1. ¿Por qué "labevents" cuelga de "hadm_id" y no de "subject_id"?*
   Debido a que un examen de laboratorio se realiza por una razón médica específica, es decir, la consulta por la que llegó al hospital. Si se utiliza subject_id entonces se podría identificar al paciente pero no se sabría a que motivo de consulta corresponden los resultados de laboratorio, ni cuanto tiempo ha estado ahí. De esa manera, se logra mantener la especificidad clínica.
2. . ¿Qué representa hadm_id clínicamente?d:
Este respresenta un encuentro clínico único, este identificador basicamente indica todo sobre el paciente desde que visitó el hospital hasta que se retiró. Es posible observar que tan serio es su problema al ver la clasificación de tratamiento.
3. *¿Por qué existe d_labitems? ¿Qué problema resuelve?*
Este es basicamente es una lista con las pruebas de laboratorio que se pueden realizar, junto con los diferentes valores que puede arrojar con sus respectivas dimensionales. Esto es útil para normalizar los dato resolviendo problemas de redundancia u optimización que se pudieran llegar a presentar; debido a que ahorra espacio.
4. *Si tuvieras una sola tabla gigante, ¿qué se volvería difícil: actualizar o consultar?*
Considero que ambos, pero uno en su mayoría. Para actualizar es necesario llegar a la dirección de los datos, por lo que estaríamos consultando. Sin embargo, al actualizar se necesita sobreescribir y si existe un archivo muy grande, entonces no sería muy óptimo para ciertos softwares hacerlo. Por ejemplo: si el paciente llamado Carlos Pérez cambia su dirección, entonces se tendría que modificar miles de filas como el registro de laboratorio, cada diagnóstico y signo vital debido a que aparece su nombre; y si se olvida una fila entonces habría inconsistencia en los datos. Además, para eso existe la herramienta de JOINS ya que al existir tablas relacionadas, la actualización en una se verá reflejada en todo el sistema...